In [1]:
import glob
import pandas as pd
import gliderport
import joblib

## all paths

In [3]:
all_paths = glob.glob('/data/female-amb/impute/10K/*.cool')

In [4]:
all_cells = [path.split('/')[-1].split('.')[0] for path in all_paths]

In [5]:
cell_table = pd.DataFrame({'cell_paths':all_paths}, index = all_cells)

In [7]:
use_cts = joblib.load('/data/metadata/m3c_use_cts')
use_cts = [ct.replace(' ','_').replace('/','') for ct in use_cts]

In [8]:
meta = pd.read_csv('/data/metadata/240104_m3C_META.csv', index_col = 0)
meta['AgingMajorType'] = meta['AgingMajorType'].apply(lambda x: x.replace(' ','_').replace('/',''))
meta = meta[meta['AgingMajorType'].isin(use_cts)]
meta['AgingMajorType.Age'] = meta['AgingMajorType'] + '.' + meta['Age']

In [10]:
cell_table['AgingMajorType.Age'] = meta['AgingMajorType.Age']

In [12]:
cell_table.to_csv('Imputed.10K.Table.csv')

## make table

In [41]:
all_table = pd.read_csv('Imputed.10K.Table.csv', index_col = 0)
all_table.head()

,cell_paths,AgingMajorType.Age
AMB_220405_8wk_11E_11F_12E_1_P1-1-B11-A1,/data/female-amb/impute/10K/AMB_220405_8wk_11E...,VLMC_NN.8wk
AMB_220405_8wk_11E_11F_12E_1_P1-1-B11-A13,/data/female-amb/impute/10K/AMB_220405_8wk_11E...,OPC_NN.8wk
AMB_220405_8wk_11E_11F_12E_1_P1-1-B11-A14,/data/female-amb/impute/10K/AMB_220405_8wk_11E...,DG_Glut.8wk
AMB_220405_8wk_11E_11F_12E_1_P1-1-B11-A2,/data/female-amb/impute/10K/AMB_220405_8wk_11E...,DG_Glut.8wk
AMB_220405_8wk_11E_11F_12E_1_P1-1-B11-B1,/data/female-amb/impute/10K/AMB_220405_8wk_11E...,L23_IT_PPP_Glut.8wk


In [49]:
all_groups = [path.split('/')[-1].split('.tsv')[0] for path in glob.glob('/data/tmp/cool_tables/*.tsv')]
len(all_groups)

120

In [50]:
for group in all_groups:
    use_cells = pd.read_csv(f"/data/tmp/cool_tables/{group}.tsv", sep = '\t', header = None)[0].tolist()
    tmpdf = all_table.loc[use_cells]
    #tmpdf.to_csv(f'loop_tables/{group}.csv', header = None)
    print(f"{group} {len(use_cells)}")

Astro-NT_NN.18mo 309
Astro-NT_NN.8wk 309
Astro-NT_NN.9mo 309
Astro-TE_NN.18mo 1412
Astro-TE_NN.8wk 1412
Astro-TE_NN.9mo 1412
CA1-ProS_Glut.18mo 501
CA1-ProS_Glut.8wk 501
CA1-ProS_Glut.9mo 501
CA3_Glut.18mo 271
CA3_Glut.8wk 271
CA3_Glut.9mo 271
CEA-BST_Gaba.18mo 414
CEA-BST_Gaba.8wk 414
CEA-BST_Gaba.9mo 414
CS-PRNr-DR_En1_Sox2_Gaba.18mo 133
CS-PRNr-DR_En1_Sox2_Gaba.8wk 133
CS-PRNr-DR_En1_Sox2_Gaba.9mo 133
DG-PIR_Ex_IMN.18mo 340
DG-PIR_Ex_IMN.8wk 340
DG-PIR_Ex_IMN.9mo 340
DG_Glut.18mo 1431
DG_Glut.8wk 1431
DG_Glut.9mo 1431
L23_IT_CTX_Glut.18mo 450
L23_IT_CTX_Glut.8wk 450
L23_IT_CTX_Glut.9mo 450
L23_IT_ENT_Glut.18mo 155
L23_IT_ENT_Glut.8wk 155
L23_IT_ENT_Glut.9mo 155
L23_IT_PPP_Glut.18mo 263
L23_IT_PPP_Glut.8wk 263
L23_IT_PPP_Glut.9mo 263
L45_IT_CTX_Glut.18mo 286
L45_IT_CTX_Glut.8wk 286
L45_IT_CTX_Glut.9mo 286
L5_ET_CTX_Glut.18mo 220
L5_ET_CTX_Glut.8wk 220
L5_ET_CTX_Glut.9mo 220
L5_IT_CTX_Glut.18mo 166
L5_IT_CTX_Glut.8wk 166
L5_IT_CTX_Glut.9mo 166
L5_NP_CTX_Glut.18mo 122
L5_NP_CTX_Glut.8w

In [47]:
#! gsutil -m cp -r loop_tables/ gs://ecker-rachel-amb-datasets/tmp

## call diff loop

In [2]:
from gliderport.preset import notebook_snakemake

In [3]:
use_cts = joblib.load('/data/metadata/m3c_use_cts')
use_cts = [ct.replace(' ','_').replace('/','') for ct in use_cts]

use_cts.remove('CA3_Glut')
use_cts.remove('Oligo_NN')
use_cts.remove('OPC_NN')

In [4]:
len(use_cts)

37

In [5]:
work_dir = 'Call.DiffLoop.CellType.Age'

In [6]:
notebook_snakemake(
    work_dir=work_dir,
    notebook_dir="template",
    groups=use_cts,
    default_cpu=10,
    default_mem_gb=30,
    redo_prepare=True,
)

In [7]:
! pwd

/home/qzeng_salk_edu/project/240213-loop
